# Trabalho Prático de Introdução a Ciência dos Dados


### Integrantes:
- Bruno Henrique Evangelista Pereira
- Francisco Teixeira Rocha Aragão
- Lorenzo Carneiro Magalhaes
- Tomas Lacerda Muniz
- Victor de Almeida Nunes Murta

##### Para mais informações, checar [README.MD](README.md)

#### **Como o moodle não suporta arquivos com mais de 20 mb, não foi possível fazer o upload com os arquivos *.csv*, devido a isso, é necessário baixá-los**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Vamos importar um dataset de tweets sobre o chatgpt, que pode ser encontrado em: [chatGPT-1000-daily-tweets](https://www.kaggle.com/datasets/edomingo/chatgpt-1000-daily-tweets)


In [ ]:
df = pd.read_csv('data/chatgpt_daily_tweets.csv')

In [ ]:
df.head()

A partir das primeiras linhas do dataframe é possível perceber que ele possui uma grande variedade de informações sobre cada tweet, mas a maioria não será utilizada em nossa análise. Então, vamos começar filtrando as informações relevantes

In [ ]:
atributos_drop = ['tweet_extracted', 'tweet_id', 'user_description', 'user_created', 'user_followers_count', 'user_following_count', 
                     'source', 'user_tweet_count', 'user_verified', 'user_id', 'user_name', 'user_username']
df = df.drop(columns=atributos_drop)
df.head()

In [ ]:
df.describe()


Agora que o dataframe já está limpo, é necessário filtrar os tweets relevantes. Para isso, vamos retirar todos os tweets que não possuem curtidas


In [ ]:
print('Tamanho original:', len(df))
df_relevante = df.query('like_count > 0')
print('Tamanho após filtragem:', len(df_relevante))

Note como a quantidade de tweets diminuiu em quase 10 vezes. Isso se deve ao fato de que boa parte dos tweets são criados por bots, que pretendemos ignorar

Uma informação que é de grande utilidade é o intervalo de datas que o dataset abrange. Vamos conferir quais são as datas do primeiro e do último tweet:

In [ ]:
df_new = df_relevante.sort_values('tweet_created')
print(df_new.iloc[0]['tweet_created'])
print(df_new.iloc[-1]['tweet_created'])

Também é interessante tentar entender como se comportam as curtidas no nosso conjunto de tweets

In [ ]:
df_relevante_copy = df_relevante.copy()
df_relevante_copy['like_count_log'] = np.log(df_relevante_copy['like_count'])

df_relevante_copy['like_count_log'].hist(bins=9, alpha=1)
plt.title('Distribuição')
plt.xlabel('Ordem de magnitude dos likes')
plt.ylabel('Frequência')
plt.show()

Agora, vamos entender qual é a distribuição das linguas dos tweets no dataframe filtrado

In [ ]:
# ordena as quantidades e tira as linguas com menos de 50 tweets
df_linguas = df_relevante.groupby('lang').count()[['text']].reset_index().rename(columns={'text': 'quantidade'})
df_linguas = df_linguas.sort_values('quantidade', ascending = True)
df_linguas = df_linguas.query('quantidade >= 50')

In [ ]:
plt.barh(df_linguas['lang'], df_linguas['quantidade'])
plt.xlabel('Idioma')
plt.ylabel('Quantidade')
plt.title('Quantidade por idioma')
plt.show()

No nosso trabalho pretendemos utilizar um algoritmo para medir a emoção associada a cada tweet sobre o chatGPT. Entretanto, o algoritmo utilizado não é capaz de fazer essa análise para linguas diferentes do inglês, então é necessário traduzir todos os tweets para o inglês antes de fazer a análise

Vamos utilizar apenas os 100 primeiros tweets já que a tradução utiliza uma api do google, então traduzir todas as linhas leva muito tempo

In [ ]:
def traduzir(tweet):
    return GoogleTranslator(source='auto', target='en').translate(tweet)

In [ ]:
df_100 = df_relevante[0:100].copy()
df_100['traduzido'] = df_100['text'].apply(traduzir)
df_100.head()

Agora que temos uma coluna traduzida, podemos criar colunas para as análises de sentimento (negativo, positivo e neutro)

In [ ]:
from src.classification.vaderFunctions import classify

# 1 for positive
# 0 for neutral
# -1 for negative

df_100['sentimental_label'] = df_100['traduzido'].apply(classify)
df_100.head()

Esses sentimentos serão explorados na versão final do projeto. O foco é entender as variações dos sentimentos em relação ao chatGPT à medida que notícias relacionadas à tecnologia eram distribuidas.

Para mais uma amostra da análise de sentimento, realizamos um teste com uma amostra gerada e classificada manualmente para verificarmos a acurácia do modelo.

In [ ]:
from src.classification.sampleVader import vader_sample_test

vader_sample_test()